In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!ls /content/drive/MyDrive/Training_Essay_Data00.csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:


import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold


# Load and preprocess the data
def load_data():
    """Load data from a CSV file with 'text' and 'label' columns."""
    df = pd.read_csv('/content/drive/MyDrive/5000human_5000machine.csv', encoding='latin1')
    texts = df['text'].tolist()
    labels = df['label'].tolist()
    return texts, labels

def preprocess_data(texts, tokenizer, max_length=128):
    """Tokenize texts and prepare labels for BERT input."""
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    return input_ids, attention_mask

# Define the BERT model
def create_model():
    """Create and compile a BERT-based text classification model."""
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')


    input_ids = tf.keras.layers.Input(shape=(128,), name='input_ids', dtype='int32')
    attention_mask = tf.keras.layers.Input(shape=(128,), name='attention_mask', dtype='int32')

    bert_output = bert_model([input_ids, attention_mask])
    cls_token = bert_output.last_hidden_state[:, 0, :]
    output = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Load BERT tokenizer
#tokenizer = TFBertTokenizer.from_pretrained('bert-base-uncased')

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess data
#file_path = 'data.csv'  # Path to your data file
texts, labels = load_data()
input_ids, attention_mask = preprocess_data(texts, tokenizer)

# Convert labels to Tensor
labels = tf.convert_to_tensor(labels)



# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold = 1
for train_index, val_index in kf.split(input_ids):
    print(f"\nFold {fold}/{k}")

    # Split data into train and validation sets
    train_input_ids, val_input_ids = tf.gather(input_ids, train_index), tf.gather(input_ids, val_index)
    train_attention_mask, val_attention_mask = tf.gather(attention_mask, train_index), tf.gather(attention_mask, val_index)
    train_labels, val_labels = tf.gather(labels, train_index), tf.gather(labels, val_index)

    # Create and compile the model
    model = create_model()
    model.summary()

    # Define callbacks for monitoring and saving the model
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'best_model_fold_{fold}.keras', save_best_only=True, monitor='val_loss')

    # Train the model
    history = model.fit(
        [train_input_ids, train_attention_mask],
        train_labels,
        validation_data=([val_input_ids, val_attention_mask], val_labels),
        epochs=5,
        batch_size=32,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Evaluate the model on the validation set
    val_predictions = model.predict([val_input_ids, val_attention_mask])
    val_predictions = (val_predictions > 0.5).astype(int)

    # Print classification report and accuracy
    print("Classification Report:\n", classification_report(val_labels, val_predictions))
    print("Accuracy Score:", accuracy_score(val_labels, val_predictions))

    fold += 1

# Save the final model
model.save('final_model.keras')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_4 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                     

250/250 [==============================] - ETA: 0s - loss: 0.1626 - accuracy: 0.9296

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


250/250 [==============================] - 333s 1s/step - loss: 0.1626 - accuracy: 0.9296 - val_loss: 0.0744 - val_accuracy: 0.9745
Epoch 2/5
250/250 [==============================] - 303s 1s/step - loss: 0.0363 - accuracy: 0.9876 - val_loss: 0.0611 - val_accuracy: 0.9820
Epoch 3/5
250/250 [==============================] - 217s 867ms/step - loss: 0.0086 - accuracy: 0.9979 - val_loss: 0.1015 - val_accuracy: 0.9685
Epoch 4/5
250/250 [==============================] - 216s 866ms/step - loss: 0.0131 - accuracy: 0.9954 - val_loss: 0.0827 - val_accuracy: 0.9710
Epoch 5/5
63/63 [==============================] - 20s 264ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.98      0.98      1020

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000

Accuracy Score: 0.9

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
